In [357]:
import pandas as pd
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [358]:
drivers = pd.read_csv('drivers.csv', header=0)
constructors = pd.read_csv('constructors.csv', header=0)
results = pd.read_csv('results.csv', header=0)
races = pd.read_csv('races.csv', header=0)

In [359]:
len(races)

1079

In [360]:
# Filer raceId here

season = list(range(1950,2022)) #Enter a F1 World Championship year
results = results[
    results['raceId'].isin(
        races[races['year'].isin(season)]['raceId'].values.tolist()
        )
    ]


### Edges

In [361]:
drivers_results = pd.merge(drivers, results, on='driverId')
constructors_results = pd.merge(constructors, results, on='constructorId')
drivers_constructors = pd.merge(drivers_results, constructors_results, on='resultId')
drivers_constructors = drivers_constructors[['driverRef', 'forename', 'surname', "dob", 'nationality_x', 'constructorRef', 'name', 'raceId_x']].drop_duplicates()
drivers_constructors.reset_index(inplace=True, drop=True)

In [362]:
dc_1 = pd.merge(drivers_constructors, drivers_constructors, on=['raceId_x', 'constructorRef'])
dc_1 = dc_1[['driverRef_x', 'driverRef_y']]
dc_1['connection'] = 'constructor'
dc_1.drop_duplicates(inplace=True)
#dc_1

In [363]:
dc_2 = pd.merge(drivers_constructors, drivers_constructors, on=['raceId_x', 'nationality_x'])
dc_2 = dc_2[['driverRef_x', 'driverRef_y']]
dc_2['connection'] = 'nation'
dc_2.drop_duplicates(inplace=True)
#dc_2

In [364]:
driver_edges = pd.concat([dc_1,dc_2])

In [365]:
edges = driver_edges[driver_edges['driverRef_x'] != driver_edges['driverRef_y']]
edges.reset_index(inplace=True, drop=True)

In [366]:
edges_list = edges.values.tolist()
edges_set = [set(dl) for dl in edges_list] 
repeat_connections = [idx for idx, ds in enumerate(edges_set) if ds in edges_set[:idx]]

In [367]:
edges.drop(repeat_connections, axis=0, inplace=True)
edges.reset_index(inplace=True, drop=True)

C:\Users\varma\AppData\Local\Temp\ipykernel_10580\1200515550.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edges.drop(repeat_connections, axis=0, inplace=True)


### Nodes

In [368]:
nodes = drivers_constructors[['driverRef', 'forename', 'surname', 'dob', 'nationality_x']]
nodes.drop_duplicates(inplace=True)
nodes.reset_index(drop=True, inplace=True)

C:\Users\varma\AppData\Local\Temp\ipykernel_10580\2592146589.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nodes.drop_duplicates(inplace=True)


### Writing to files

In [369]:
edges.to_csv('./data/f1_edges_1950-21.csv', index = False)
nodes.to_csv('./data/f1_nodes_1950-21.csv', index = False)

In [370]:
nodes
edges

,driverRef,forename,surname,dob,nationality_x
0,hamilton,Lewis,Hamilton,1985-01-07,British
1,heidfeld,Nick,Heidfeld,1977-05-10,German
2,rosberg,Nico,Rosberg,1985-06-27,German
3,alonso,Fernando,Alonso,1981-07-29,Spanish
4,kovalainen,Heikki,Kovalainen,1981-10-19,Finnish
...,...,...,...,...,...
845,russell,George,Russell,1998-02-15,British
846,albon,Alexander,Albon,1996-03-23,Thai
847,latifi,Nicholas,Latifi,1995-06-29,Canadian
848,pietro_fittipaldi,Pietro,Fittipaldi,1996-06-25,Brazilian


,driverRef_x,driverRef_y,connection
0,hamilton,kovalainen,constructor
1,hamilton,alonso,constructor
2,hamilton,button,constructor
3,hamilton,rosberg,constructor
4,hamilton,bottas,constructor
...,...,...,...
11181,lof,flinterman,nation
11182,perez,gutierrez,nation
11183,stroll,latifi,nation
11184,norris,aitken,nation
